# Antenna Pattern Test @ 141.7 Hz

Predice la relación de amplitudes esperada H1/L1/V1 para un evento dado (RA, Dec, pol) y compara con la observada.

In [ ]:
import numpy as np
from math import sin, cos, pi

# --- Detector site geometry (lat [rad], lon [rad], arm azimuths [rad]) ---
deg = pi/180.0
SITES = {
  'H1': {'lat': 46.455*deg, 'lon': -119.408*deg, 'gamma': 171.8*deg},
  'L1': {'lat': 30.563*deg, 'lon':  -90.774*deg, 'gamma': 243.0*deg},
  'V1': {'lat': 43.630*deg, 'lon':   10.505*deg, 'gamma': 116.5*deg}
}

def rot_z(a):
    c,s = np.cos(a), np.sin(a)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]])

def sky_to_enu(ra, dec, gmst, lat, lon):
    # unit vector in ECI coords
    nx = np.cos(dec)*np.cos(ra)
    ny = np.cos(dec)*np.sin(ra)
    nz = np.sin(dec)
    n = np.array([nx, ny, nz])
    # rotate by GMST to ECEF, then to local ENU at site
    R_g = rot_z(gmst)
    ecef = R_g @ n
    # local basis
    sl, cl = np.sin(lat), np.cos(lat)
    so, co = np.sin(lon), np.cos(lon)
    # ECEF->ENU
    T = np.array([[-so,  co, 0],
                  [-cl*co, -cl*so, sl],
                  [ sl*co,  sl*so, cl]])
    enu = T @ ecef
    return enu/np.linalg.norm(enu)

def det_tensor(gamma):
    # Interferometer arms u,v rotated by gamma in local EN plane
    ug = np.array([ np.cos(gamma),  np.sin(gamma), 0.0])
    vg = np.array([-np.sin(gamma),  np.cos(gamma), 0.0])
    return 0.5*(np.outer(ug,ug)-np.outer(vg,vg))

def pol_tensors(nhat, psi):
    # choose an orthonormal basis (p,q,nhat) with p in tangent plane
    z = np.array([0,0,1.0])
    if abs(np.dot(nhat,z))>0.9:
        z = np.array([0,1,0])
    p = np.cross(nhat, z); p = p/np.linalg.norm(p)
    q = np.cross(nhat, p)
    # rotate by polarization angle psi around nhat
    cp, sp = np.cos(psi), np.sin(psi)
    p2 =  cp*p + sp*q
    q2 = -sp*p + cp*q
    eplus  = np.outer(p2,p2) - np.outer(q2,q2)
    ecross = np.outer(p2,q2) + np.outer(q2,p2)
    return eplus, ecross

def Fplus_Fcross(ra, dec, psi, gmst, site):
    lat, lon, gamma = site['lat'], site['lon'], site['gamma']
    nh = sky_to_enu(ra, dec, gmst, lat, lon)
    d  = det_tensor(gamma)
    eplus, ecross = pol_tensors(nh, psi)
    Fp = (d*eplus).sum(); Fx = (d*ecross).sum()
    return Fp, Fx

# Example inputs (replace with event values)
ra   = 3.0   # radians
dec  = -0.5  # radians
psi  = 0.2   # radians (polarization)
gmst = 1.0   # radians (GMST at coalescence)

res = {}
for det in ['H1','L1','V1']:
    Fp, Fx = Fplus_Fcross(ra,dec,psi,gmst,SITES[det])
    A = np.sqrt(Fp**2 + Fx**2)
    res[det] = {'Fp':Fp,'Fx':Fx,'A':A}
res

### Ratio esperado de amplitudes
Usa `res['H1']['A']`, etc., y compáralo con la amplitud observada a 141.7 Hz por detector.